In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import pyreadr
import os 


$$
\begin{aligned}
C_i \frac{d T_i}{d t}= & \frac{1}{R_a}\left(T_a-T_i\right)+\frac{1}{R_i}\left(T_{i m}-T_i\right) \\
& +\frac{1}{R_{i m}}\left(T_{o m}-T_i\right)+A(1-p) \Phi_s \\
& +\Phi_h \\
C_{i m} \frac{d T_{i m}}{d t}= & \frac{1}{R_i}\left(T_i-T_{i m}\right)+A p \Phi_s \\
C_{o m} \frac{d T_{o m}}{d t}= & \frac{1}{R_{i m}}\left(T_i-T_{o m}\right)+\frac{1}{R_{a m}}\left(T_a-T_{o m}\right)
\end{aligned}
$$

In [3]:
Ri = 46.848    # Resistance between indoor air and interior walls
Rim = 1.809    # Resistance between indoor air and house envelop
Ra = 8.021     # Resistance between indoor air and outdoor
Ram = 0.0846   # Resistance between house envelope and outdoor

Ci = 8.120      # heat capacity of indoor air
Cim = 0.00553   # heat capacity of interior walls
Com = 292.905   # heat capacity of house envelope

A = 20.212     # window area
p = 1          # fraction

If,

$$
\frac{d \boldsymbol{T}}{d t}=A T+B U
$$

Now we can identify the matrices $A$ and $B$ :
Now we can identify the matrices $A$ and $B$ :
$$
A=\left[\begin{array}{ccc}
-\frac{1}{C_i}-\frac{1}{C_i R_a}-\frac{1}{C_i R_i}-\frac{1}{C_i R_{i m}} & 0 & 0 \\
\frac{1}{C_{i \mathrm{im}} R_i} & -\frac{1}{C_{i m} R_i} & 0 \\
C_{o m} R_{i m} & 0 & -\frac{1}{C_{o m} R_{i m}}-\frac{1}{C_{o m} R_{a m}}
\end{array}\right]
$$

$$
B=\left[\begin{array}{ccc}
\frac{1}{C_iR_a} & \frac{A(1-p)}{C_i} & \frac{1}{C_i} \\
0 & \frac{A p}{C_{\mathrm{im}}} & 0 \\
\frac{1}{C_{o m} R_{\mathrm{am}}} & 0 & 0
\end{array}\right]
$$

In [4]:
# Construct matrix A
A_matrix = np.array([
    [-1 / (Ci * Ra) - 1 / (Ci * Ri) - 1 / (Ci * Rim), 1 / (Ci * Ri), 1 / (Ci * Rim)],
    [1 / (Cim * Ri), -1 / (Cim * Ri), 0],
    [1 / (Com * Rim), 0, -1 / (Com * Rim) - 1 / (Com * Ram)]
])

# Construct matrix B
B_matrix = np.array([
    [1 / (Ci * Ra), A * (1 - p) / Ci, 1 / Ci],
    [0, A * p / Cim, 0],
    [1 / (Com * Ram), 0, 0]
])

Create input data for each time point to solve the differential equation

In [5]:
path = os.getcwd()

In [6]:
os.getcwdb()

b'c:\\Users\\Dell\\Documents\\GitHub\\PINNS_OPF\\opf\\notebooks'

In [7]:

result = pyreadr.read_r(path=r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\opf\data\raw\Exercise3.RData')
result.keys() # extract the pandas data frame for object df1

odict_keys(['AllDat'])

In [8]:
input_df = result['AllDat']
input_df.columns

Index(['date', 't', 'yTi1', 'yTi2', 'yTi3', 'yTi4', 'Ta', 'Gv', 'Ph1', 'Ph2'], dtype='object')

In [9]:
U = np.array([input_df['Ta'].to_numpy(),input_df['Gv'].to_numpy(),input_df['Ph1'].to_numpy()])

In [76]:
Sigma = np.diag([np.exp(-2.754),np.exp(2.368),np.exp(-0.943)])
Sigma

array([[ 0.06367266,  0.        ,  0.        ],
       [ 0.        , 10.67601888,  0.        ],
       [ 0.        ,  0.        ,  0.38945771]])

In [77]:
np.random.multivariate_normal(mean=[0,0,0],cov=Sigma)

array([ 0.12229158, -1.59722931,  0.04727038])

In [80]:
# Modify the differential equation function to include U
def differential_equation(T, t, A, B, U):
    U = U[:,int(t)]
    dTdt = np.dot(A, T) + np.dot(B, U) 
    return dTdt

initial_Ti = 15
initial_Tim = 17
initial_Tom = 12

# Define the initial conditions
T_initial = np.array([initial_Ti, initial_Tim, initial_Tom])

# Define the time points
t = t = np.linspace(1,100,1200)

# Solve the differential equation
solution = odeint(differential_equation, T_initial, t, args=(A_matrix, B_matrix, U))

# Extract the solution components
Ti_solution = solution[:, 0]
Tim_solution = solution[:, 1]
Tom_solution = solution[:, 2]

In [81]:
Ti_solution

array([15.        , 15.08713937, 15.18316765, ..., 15.91050706,
       15.97850391, 16.04621649])